In [138]:
# imports

import numpy as np
import os
import yaml, sys
try:
    import ruamel_yaml as ry
except Exception:
    try:
        import ruamel.yaml as ry
    except Exception:
        raise ImportError('No module named ruamel.yaml or ruamel_yaml')
        

import warnings
# warnings.simplefilter('ignore', ry.yaml.error.MantissaNoDotYAML1_1Warning)


In [139]:
# Helper functions
def get_line_type(orca_yaml,type_search):
    for line_type in orca_yaml['LineTypes']:
        if line_type['Name'] == type_search:
            break
    return line_type

def get_variable_info(orca_yaml,variable_type, variable_name):
    for var in orca_yaml['VariableData'][variable_type]:
        if var['Name'] == variable_name:
            break
    return var['IndependentValue, DependentValue']



In [140]:
from wisdem.inputs.validation import load_yaml

orca_input = '/Users/dzalkind/Downloads/K03 15MW semi-sub FOWT.yml'
orca_yaml = load_yaml(orca_input)


# Tower

In [141]:
platform_name = 'VolturnUS-S'

weis_tower = {}
weis_tower['outer_shape_bem'] = {}
weis_tower['internal_structure_2d_fem'] = {}



# Tower line objet
is_tower_line = [(line['Name'] == 'Tower') for line in orca_yaml['Lines']]
if not any(is_tower_line):
    raise Exception('No OrcaFlex Line objects are named "Tower".  This is required for WindIO tower information.')

tower_line = orca_yaml['Lines'][np.where(is_tower_line)[0][0]]

# Platform connection
connections = tower_line['Connection, ConnectionX, ConnectionY, ConnectionZ, ConnectionAzimuth, ConnectionDeclination, ConnectionGamma, ConnectionReleaseStage, ConnectionzRelativeTo']
# TODO: will we need to parse these multiple entries better?

is_platform_connection = [(con[0] == platform_name) for con in connections]
platform_connection = connections[np.where(is_platform_connection)[0][0]]
platform_connection_xyz = platform_connection[1:4]   # TODO: parse better later?
platform_connection_xyz

# sections = []  # Supporting only 1 section for now

tower_line_types = tower_line['LineType, Length, TargetSegmentLength']  # These are sections in orcaflex.  Not sure how we'll handle multiple. Save that for later
previous_section_height = platform_connection_xyz[2]

section = {}
section['type'] = line_type[0]
section['length'] = line_type[1]
type_info = get_line_type(orca_yaml,section['type'])


outer_diameter = np.array(get_variable_info(orca_yaml,'Linetypediameter','Tower OD'))  # top is 0
inner_diameter = np.array(get_variable_info(orca_yaml,'Linetypediameter','Tower ID'))  # top is 0



# It's an open question about what should be the ref axis.  Since we're sharing a reference with the inner and outer diameter, we'll use that as the reference axis for now.
# Alternative: section['z'] = np.arange(previous_section_height,line_type[1],line_type[2])
# Usign the inner/outer diameter variable info matches with the Also it matches with the IEA-15 WindIO input
# If we get different values from partners, we will want to map/interpolate to a different axis
ref_axis = {}
ref_axis['x'] = {}
ref_axis['y'] = {}
ref_axis['z'] = {}

ref_axis['z']['values'] = outer_diameter[:,0] + platform_connection_xyz[2]
ref_axis['z']['grid'] = (ref_axis['z']['values'] - ref_axis['z']['values'][0]) / (ref_axis['z']['values'][-1] - ref_axis['z']['values'][0])

# X and Y follow for now
ref_axis['x']['grid'] = ref_axis['y']['grid'] = ref_axis['z']['grid']
ref_axis['x']['values'] = ref_axis['y']['values'] = np.zeros(len(ref_axis['z']['grid']))
weis_tower['outer_shape_bem']['reference_axis'] = ref_axis


weis_tower['outer_shape_bem']['outer_diameter'] = {}
weis_tower['outer_shape_bem']['outer_diameter']['grid'] = ref_axis['z']['grid']  # Shared for now, doesn't necessarily have to
weis_tower['outer_shape_bem']['outer_diameter']['values'] = np.flip(outer_diameter[:,1])  # It's not super clear what indicator OrcaFlex uses to flip these coordinates, but they definitely are flipped

# drag coefficient
weis_tower['outer_shape_bem']['drag_coefficient'] = {}
weis_tower['outer_shape_bem']['drag_coefficient']['grid'] = [0,1]
weis_tower['outer_shape_bem']['drag_coefficient']['values'] = 2*[type_info['Cdn']] # TODO: double check this one

# Internal structure
weis_tower['internal_structure_2d_fem']['reference_axis'] = ref_axis
weis_tower['internal_structure_2d_fem']['outfitting_factor'] = 1.07 # TODO: figure out if OrcaFlex is doing this
layers = []
tower_layer = {}
tower_layer['name'] = 'tower_wall'
tower_layer['material'] = 'tower_material'   # TODO: fill this in from available OrcaFlex info
tower_layer['thickness'] = {}
tower_layer['thickness']['grid'] = ref_axis['z']['grid']


diameter_diff = (outer_diameter - inner_diameter)  #[:,1]
thickness = np.flip(diameter_diff[:,1] / 2)

tower_layer['thickness']['values'] = thickness

tower_material = {}




In [128]:
# Find line type
# Look for the name of a type and return the first version

tower_material = {
  'name': 'tower_material'  ,
  'description': f'Information pulled from OrcaFlex input {orca_input}' , 
  'source': f'Information pulled from OrcaFlex input {orca_input}'   ,
  'orth': 0, # TODO: Check
  'rho': type_info['MaterialDensity'] * 1000,  # includes outfitting_factor in WindIO, OrcaFlex in g/cm^3
  'alpha': 'temp'  , # TODO: Check
  'E': type_info['E']  , 
  'nu': 'temp'  , 
  'G': 'temp'  ,
  'GIc': 'temp'  ,
  'GIIc': 'temp'  ,
  'alp0': 'temp'  ,
  'Xt': 'temp'  ,
  'Xc': 'temp'  ,
  'S': 'temp'  ,
  'Xy': 'temp'  ,
  'm': 'temp'  ,
  'A': 'temp'  ,
  'unit_cost': 'temp'  ,
}


# orca_yaml['LineTypes']

In [133]:
type_info['E']

200000000.0